In [26]:
pip install annoy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp312-cp312-macosx_10_12_universal2.whl size=112740 sha256=66374e615f043042b27a95f146b7a222b003838b47b9b05b46576d35ffb27067
  Stored in directory: /Users/soumyadip_iitk/Library/Caches/pip/wheels/db/b9/53/a3b2d1fe1743abadddec6aa541294b24fdbc39d7800bc57311
Successfully built annoy
Note: you may need to restart the kernel to use updated packages.


In [1]:
from importlib.metadata import metadata
import torch
import torchvision
from torchvision import models, transforms
import os
from PIL import Image, ImageDraw
import torch.nn as nn
from annoy import AnnoyIndex

In [61]:
images_file = '/Users/soumyadip_iitk/PycharmProjects/<231026>_<Soumyadip>_deeploycv/untitled folder/Dog'
images = os.listdir(images_file)
print(len(images))

12499


In [62]:
weights = models.ResNet18_Weights.IMAGENET1K_V1
model = models.resnet18(weights=weights)
model.fc = nn.Identity()
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [63]:
model.eval()
transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])


In [64]:
annoy_index = AnnoyIndex(512,'angular')

In [65]:
for i in range(len(images)):
    image = Image.open(os.path.join(images_file, images[i]))
    input_tensor = transform(image).unsqueeze(0)

    if input_tensor.size()[1] == 3:
        output_tensor = model(input_tensor)
        # print(f'{i} predicted as {output_tensor.size()} predicted as {weights.meta["categories"][torch.argmax(output_tensor)]}')
        # input()
        #print(output_tensor.shape())
        annoy_index.add_item(i, output_tensor[0])
        if i%100==0 & i!=11702:
            print(f'We have processed {i} images.')

annoy_index.build(10) #10 number of trees
annoy_index.save('dog.ann')

We have processed 0 images.
We have processed 100 images.
We have processed 200 images.
We have processed 300 images.
We have processed 400 images.
We have processed 500 images.
We have processed 600 images.
We have processed 700 images.
We have processed 800 images.
We have processed 900 images.
We have processed 1000 images.
We have processed 1100 images.
We have processed 1200 images.
We have processed 1300 images.
We have processed 1400 images.
We have processed 1500 images.
We have processed 1600 images.
We have processed 1700 images.
We have processed 1800 images.
We have processed 1900 images.
We have processed 2000 images.
We have processed 2100 images.
We have processed 2200 images.
We have processed 2300 images.
We have processed 2400 images.
We have processed 2500 images.
We have processed 2600 images.
We have processed 2700 images.
We have processed 2800 images.
We have processed 2900 images.
We have processed 3000 images.
We have processed 3100 images.
We have processed 32

True

***Building Annoy Index***

In [71]:
image_grid = Image.new('RGB', (1000,1000))
for i in range(len(images)):
    image = Image.open(os.path.join(images_file, images[i]))
    input_tensor = transform(image).unsqueeze(0)

    if input_tensor.size()[1] == 3:
        output_tensor = model(input_tensor)
        nns = annoy_index.get_nns_by_vector(output_tensor[0], 24)
        image = image.resize((200,200))
        image_draw = ImageDraw.Draw(image)
        image_draw.rectangle([(0,0),(199,199)],outline='red',width=8)
        image_grid.paste(image,(0,0))

        for j in range(24):
            search_image = Image.open(os.path.join(images_file, images[nns[j]]))
            search_image = search_image.resize((200,200))
            image_grid.paste(search_image,((200*((j+1)%5),200*((j+1)//5))))

    image_grid.save(f'Image_dump/image_{i}.png')

KeyboardInterrupt: 